Teoretiska Frågor
1. Beskriv kort hur en relationsdatabas fungerar.

    En relationsdatabas organiserar data i "tables" som består av rader och kolumner. Primary keys och foreign keys (likt python dictionaries key-value) används för att hänvisa till olika tables.

2. Vad menas med “CRUD”-flödet?

    "CRUD"-flödet står för de operationer som man kan göra i sin data:
    Create, Read, Update och Delete

3. Beskriv kort vad en “left join” och en “inner join” är. Varför använder man det?

    Left join används för att retunera alla rader i ett table samt matchande rader i ett annat table. Man kan se det som alla element i A i ett klassiskt venn-diagram.
    Inner join blir A snitt B, alltså retuneras alla rader som matchar både i första (left) och andra (right) tabellen.
    
4. Beskriv kort vad indexering i SQL innebär.
    Indexering skapar ytterligare data struktur som hänvisar till
5. Beskriv kort vad en vy i SQL är.
6. Beskriv kort vad en lagrad procedur i SQL är.